In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


We now need to find patterns in the training data to make predictions for the test data.

In [4]:
#let's start with gender

women = train_data.loc[train_data.Sex == "female"]["Survived"]
rate_women = sum(women)/len(women)

print(" % of women survived: ", rate_women*100)


# print(" % of men survived: ", (1-rate_women)*100)
# wrong approach- survival of men and women is independent

 % of women survived:  74.20382165605095


In [5]:
men = train_data.loc[train_data.Sex == "male"]["Survived"]
rate_men = sum(men)/len(men)

print(" % of men survived: ", rate_men*100)


 % of men survived:  18.890814558058924


74% women survived, and only 18% men survived. Hence gender as a factor for survival was a good guess.

But now, considering various different columns for more complex patterns is difficult as well as time consuming, we'll let **ML** automate this process for us

We will be using the **Random Forest Algorithm** to make several decision trees using different column parameters to make distinct trees. Then, each of these trees will give an output of survival chance, and a collective decision will be made based on voting

In [6]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]

# we now need to convert all our categorical features data into numeric binary format 
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])



In [7]:
# we want to make 100 different decision trees with PnC of the features, each having a maximum depth
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

model.fit(X,y)

RandomForestClassifier(max_depth=5, random_state=1)

In [8]:
predictions = model.predict(X_test)

output = pd.DataFrame({
    'PassengerID' : test_data.PassengerId,
    'Survived' : predictions
})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
